In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import mdtraj
import pytraj as pt
from simtk import openmm, unit


In [2]:

#BGangles = np.loadtxt("HNCanglesBG.dat", usecols=1)
#MDangles = np.loadtxt("HNCanglesMD.dat", usecols=1)

def plot_dof_comparison(BGdof, MDdof, MD2dof, dofname):  
    fig, ax = plt.subplots()    

    if 'bond' in dofname:
        ax.set_xlabel(f"{dofname} ($\AA$)")
    if 'angle' in dofname:
        ax.set_xlabel(f"{dofname} ($^\circ$)")
    if 'dihed'in dofname:
        ax.set_xlabel(f"{dofname} ($^\circ$)")
    # y-axis on the right
    
    ax.hist(BGdof, bins=40, label="BG", alpha=1)
    ax.hist(MDdof, bins=40, label="MD", alpha=0.7)
    ax.hist(MD2dof, bins=40, label="MD2", alpha=0.5)
    ax.set_ylabel(f"Count   [#{dofname} / {len(MDdof)}]")
    ax.legend()
    plt.savefig(f"Histograms/{dofname}.png")
    plt.close

In [ ]:
#plot_dof_comparison(BGangles,MDangles,"HNC angle")

In [ ]:
trajectory_bg = pt.load('samplestraj2000mixed.pdb', top='test.prm7')
trajectory_md = pt.load('mdtraj.pdb',top= 'test.prm7',stride=100)
trajectory_md2 = pt.load('output.pdb', top = 'test.prm7')

In [ ]:
def calc_bonds(trajectory,type):
    bonds = {}
    for bond in trajectory.topology.bonds:
        bonds[str(bond.indices)] = pt.calc_distance(trajectory, bond.indices)[0]
    df = pd.DataFrame(bonds)
    #df.to_csv(f'{type}bonds.csv')
    return df
    

In [ ]:
headers = []
atompairs = np.empty((trajectory_md2.topology.n_bonds,2))
i = 0
for bond in trajectory_md2.topology.bonds:
    headers.append(str(bond))
    atompairs[i,:] = [bond.atom1.index,bond.atom2.index]
    i += 1
#print(atompairs[0:2,:])
MD2bonds = mdtraj.compute_distances(trajectory_md2,atompairs)
#print(MDbonds)

In [ ]:
MD2bondsdf = pd.DataFrame(MD2bonds, columns=[headers])

In [ ]:
for col in MD2bondsdf.columns:
    fig, ax = plt.subplots()   
    
    ax.set_xlabel(f"length ($\AA$)")
    ax.hist((MD2bondsdf[col]*10), bins=40, label="MD",)
    ax.set_ylabel(f"Count   [#bonds / {len(MD2bondsdf[col])}]")
    ax.legend()
    #plt.savefig(f"Histograms/{dofname}.png" )
    plt.close

In [ ]:
plt.close('all')

In [ ]:
BGbonds = calc_bonds(trajectory_bg, type='BG')
MDbonds = calc_bonds(trajectory_md, type='MD')
MD2bonds = calc_bonds(trajectory_md2, type = 'MD2')
print(BGbonds.columns,MDbonds.columns,MD2bonds.columns)

In [ ]:
#print(BGbonds.columns)
for col in BGbonds.columns:
    plot_dof_comparison(BGbonds[col],MDbonds[col],MD2bonds[col] ,dofname=f'bond_{col}')

In [ ]:
test = pt.load('samplestraj2000mixed.pdb', top='ala.prm7')

In [ ]:
def calc_diheds(trajectory,type):
    diheds = {}
    for dihedral in trajectory.topology.dihedrals:
        #print(dihedral)
        diheds[str(dihedral.indices)] = pt.calc_dihedral(trajectory, dihedral.indices)[0]

    df = pd.DataFrame(diheds)
    df.to_csv(f'{type}diheds.csv')
    return df


In [ ]:
BGdiheds = calc_diheds(trajectory_bg,'BG')
MDdiheds = calc_diheds(trajectory_md,'MD')


In [ ]:
#print(BGdiheds.columns)
for col in BGdiheds:
    plot_dof_comparison(BGdiheds[col],MDdiheds[col], dofname=f'dihed_{col}')
#plot_dof_comparison(BGdiheds['[5 4 6 8]'],MDdiheds['[5 4 6 8]'], dofname='dihed_[5 4 6 8]')

In [ ]:
anglemasks = pd.read_csv('ala_angles.txt',delim_whitespace=True,usecols=[3,4,5,6,7,8])

anglemasks['full'] = anglemasks[['Atom1','Atom2','Atom3']].agg(" ".join, axis=1)
anglemasks['indices'] = anglemasks[['A1','A2','A3']].astype(str).agg(" ".join, axis=1)
#(" ".join, axis=1)
print(anglemasks['indices'])

In [ ]:
def calc_angles(trajectory,type):
    angles = {}
    for maskset in anglemasks['full']:
        #print(maskset)
        angles[maskset] = pt.calc_angle(trajectory, str(maskset))

    df = pd.DataFrame(angles,)
    df.to_csv(f'{type}angles.csv')
    return df


In [ ]:
print(anglemasks['full'][1])
pt.calc_angle(trajectory_bg, anglemasks['full'][1])

anglesBG = calc_angles(trajectory_bg, 'BG')
anglesMD = calc_angles(trajectory_md, type='MD')

In [ ]:
for col in anglesBG:
    plot_dof_comparison(anglesBG[col],anglesMD[col], dofname=f'angle_{col}')